In [2]:
import numpy as np
from tensorflow import keras
from keras import layers

In [3]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

In [5]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 16s 37ms/step - loss: 0.3683 - accuracy: 0.8883 - val_loss: 0.0833 - val_accuracy: 0.9780
Epoch 2/15
422/422 [==============================] - 16s 38ms/step - loss: 0.1115 - accuracy: 0.9650 - val_loss: 0.0558 - val_accuracy: 0.9847
Epoch 3/15
422/422 [==============================] - 17s 41ms/step - loss: 0.0857 - accuracy: 0.9740 - val_loss: 0.0500 - val_accuracy: 0.9865
Epoch 4/15
422/422 [==============================] - 18s 44ms/step - loss: 0.0713 - accuracy: 0.9776 - val_loss: 0.0432 - val_accuracy: 0.9878
Epoch 5/15
422/422 [==============================] - 19s 45ms/step - loss: 0.0643 - accuracy: 0.9802 - val_loss: 0.0374 - val_accuracy: 0.9903
Epoch 6/15
422/422 [==============================] - 20s 48ms/step - loss: 0.0566 - accuracy: 0.9820 - val_loss: 0.0361 - val_accuracy: 0.9912
Epoch 7/15
422/422 [==============================] - 21s 51ms/step - loss: 0.0522 - accuracy: 0.9838 - val_loss: 0.0343 - val_accuracy:

In [6]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.025210143998265266
Test accuracy: 0.991599977016449


In [7]:
import pickle
saved_model = pickle.dumps(model)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\conv2d
......vars
.........0
.........1
...layers\conv2d_1
......vars
.........0
.........1
...layers\dense
......vars
.........0
.........1
...layers\dropout
......vars
...layers\flatten
......vars
...layers\max_pooling2d
......vars
...layers\max_pooling2d_1
......vars
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-04-07 16:50:58         2753
metadata.json                                  2023-04-07 16:50:58           64
variables.h5                                   2023-04-07 16:50:58       447568


In [8]:
model_from_pickle = pickle.loads(saved_model)

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-04-07 16:50:58         2753
metadata.json                                  2023-04-07 16:50:58           64
variables.h5                                   2023-04-07 16:50:58       447568
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\conv2d
......vars
.........0
.........1
...layers\conv2d_1
......vars
.........0
.........1
...layers\dense
......vars
.........0
.........1
...layers\dropout
......vars
...layers\flatten
......vars
...layers\max_pooling2d
......vars
...layers\max_pooling2d_1
......vars
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars


In [15]:
# predicting a new image
image = x_test[5]
prediction = model_from_pickle.predict(image.reshape(1, 28, 28, 1))
labels=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
print(labels[np.argmax(prediction)])
print(labels[np.argmax(y_test[5])])
import cv2 
cv2.imshow("image", cv2.cvtColor(x_test[5], cv2.COLOR_BGR2RGB))
cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 [==============================] - 0s 18ms/step
1
1
